In [1]:
### DISTRIBUTION OF DISORDERS BY PREFERENTIAL PARENT


# We launch the config file with the installation of the R libraries
source("config.R")


# INPUTS : "product1_en_cross_jdbor_evo.xml"; "product7_linear_en_jdbor_evo.xml".


# OUTPUTS : "distribution_disorders_by_preferential_parent".

In [2]:
# We launch the libraries
library(tidyverse)
library(rlist)
library(data.table)
library(dplyr)
library(XML)
library(methods)

Installing package into 'C:/Users/ohongnat/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpET2dNz\downloaded_packages


Installing package into 'C:/Users/ohongnat/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'rlist' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpET2dNz\downloaded_packages


Installing package into 'C:/Users/ohongnat/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


...

Installing package into 'C:/Users/ohongnat/Documents/R/win-library/3.6'
(as 'lib' is unspecified)


package 'XML' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'XML'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problème lors de la copie de C:\Users\ohongnat\Documents\R\win-library\3.6\00LOCK\XML\libs\x64\XML.dll vers C:\Users\ohongnat\Documents\R\win-library\3.6\XML\libs\x64\XML.dll: Permission denied"Warning message:
"restored 'XML'"


The downloaded binary packages are in
	C:\Users\ohongnat\AppData\Local\Temp\RtmpET2dNz\downloaded_packages


Installing package into 'C:/Users/ohongnat/Documents/R/win-library/3.6'
(as 'lib' is unspecified)
Warning message:
"package 'methods' is not available (for R version 3.6.1)"Warning message:
"package 'methods' is a base package, and should not be updated"Warning message:
"package 'tidyverse' was built under R version 3.6.3"-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --
v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.0     v dplyr   1.0.6
v tidyr   1.1.3     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'readr' was built under R version 3.6.3"Warning message:
"package 'purrr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"Warning mess

In [3]:
### Later we will need to get the number of total disorders for our calculations

# We open xml "product1_en_cross_jdbor_evo" that contains these datas
product1=xmlParse("product1_en_cross_jdbor_evo.xml")
product1=xmlToList(product1)

# We remove the last element of the list, which is the numeric count of all elements of the list
product1=product1$DisorderList
product1=product1[-length(product1)]

### Our list for working
indice_list=1
product=list()
for (i in 1:length(product1))
{
  product[[indice_list]]=product1[[i]]
  indice_list=indice_list+1
}
# Each element in the list is a disorder

In [4]:
# We only select the active clinical entities:
# Head of classification (flag=128), Historical entities (flag=512) & On-line (flag=1)
indice_list=1
all_active_clinical_entities=list()
for (i in 1:length(product))
  if (product[[i]]$DisorderFlagList$DisorderFlag$Value=='1' |
      product[[i]]$DisorderFlagList$DisorderFlag$Value=='128' |
      product[[i]]$DisorderFlagList$DisorderFlag$Value=='512')
  {
    all_active_clinical_entities[[indice_list]]=product[[i]]
    indice_list=indice_list+1
  }

In [5]:
### Number of disorders
indice_list=1
disorders=list()
for (i in 1:length(all_active_clinical_entities))
  if (all_active_clinical_entities[[i]]$DisorderGroup$Name$text=='Disorder')
  {
    disorders[[indice_list]]=all_active_clinical_entities[[i]]
    indice_list=indice_list+1
  }

In [6]:
# We keep the OrphaCodes of the disorders
orphacode_disorders=list()
for (i in 1:length(disorders))
{
  orphacode_disorders[[i]]=data.frame(disorders[[i]]$OrphaCode)
}
orphacode_disorders=rbindlist(orphacode_disorders)
names(orphacode_disorders)=c('OrphaCode')

In [7]:
# We can now work on preferential parents
# We select the xml with the datas we need
xmlfiles = list.files(pattern='product7')
product7=xmlToList(xmlfiles)

# Our list for working
product7=product7$DisorderList
indice_list=1
product7_list=list()
for (i in 1:length(product7))
{
  product7_list[[indice_list]]=product7[[i]]
  indice_list=indice_list+1
}
# Each element in the list is a disorder

In [8]:
# We keep only the disorders which have a preferential parent
indice_list=1
disorders_with_preferential_parent=list()
for (i in 1:(length(product7_list)-1))
  if ('TargetDisorder' %in% names(product7_list[[i]]$DisorderDisorderAssociationList$DisorderDisorderAssociation))
  {
    disorders_with_preferential_parent[[indice_list]]=data.frame(product7_list[[i]]$OrphaCode, 
                                                                 product7_list[[i]]$DisorderDisorderAssociationList$DisorderDisorderAssociation$TargetDisorder$Name$text)
    indice_list=indice_list+1
  }
disorders_with_preferential_parent=rbindlist(disorders_with_preferential_parent,use.names = TRUE)
names(disorders_with_preferential_parent)=c('OrphaCode', 'Preferential parent')

In [9]:
# We keep only those which are active disorders
active_preferential_parent=merge(orphacode_disorders, disorders_with_preferential_parent,
                                 by='OrphaCode', all.x = TRUE)

In [10]:
### Distribution of preferential parents
distribution_disorders_by_preferential_parent=as.data.frame(table(active_preferential_parent$`Preferential parent`))
names(distribution_disorders_by_preferential_parent)=c('Preferential parent','Distribution')
distribution_disorders_by_preferential_parent

Preferential parent,Distribution
Rare bone disease,351
Rare neurologic disease,1028
Rare inborn errors of metabolism,384
Rare developmental defect during embryogenesis,2000
Rare hematologic disease,171
Rare skin disease,383
Rare odontologic disease,15
Rare cardiac disease,47
Rare infectious disease,160
Rare respiratory disease,75
